<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#B1-能量沉积" data-toc-modified-id="B1-能量沉积-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>B1-能量沉积</a></span><ul class="toc-item"><li><span><a href="#回顾" data-toc-modified-id="回顾-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>回顾</a></span></li><li><span><a href="#B1中通过Action-class实现获取能量沉积" data-toc-modified-id="B1中通过Action-class实现获取能量沉积-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>B1中通过Action class实现获取能量沉积</a></span></li></ul></li><li><span><a href="#B1-多线程" data-toc-modified-id="B1-多线程-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>B1-多线程</a></span><ul class="toc-item"><li><span><a href="#B1的多线程是什么意思？" data-toc-modified-id="B1的多线程是什么意思？-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>B1的多线程是什么意思？</a></span></li></ul></li></ul></div>

## B1-能量沉积

### 回顾
- run, event, step, track
  - 一个run会有多个event
  - 每个event会有多个step，直至粒子消亡
  - 一个event的能量沉积相当于多个step的能损之和
  - 每个step的信息通过track获取
  - 进而可以通过循环求和得到粒子在介质中的能损
  
### B1中通过Action class实现获取能量沉积
- B1RunAction
  - 继承了G4UserRunAction
  - BeginOfRunAction()
  - EndOfRunAction()
  - AddEdep()
- B1EventAction
  - 继承了G4UserEventAction
  - BeginOfEvnetAction()
  - EndOfEventAction()
  - AddEdep() 
- B1SteppingAction
  - 继承了G4UserSteppingAction
  - UserSteppingAction()
    - 判断粒子是否在logicvolume中：step->GerPreSetPoint()->GetTouchableHandle()->GetVolume()->GetLogicalVolume();
- 注意
  - BeginOfRunAction() vs BeginOfEvnetAction()
    - BeginOfRunAction()是一个run调用一次
    - BeginOfEvnetAction()是一个Event调用一次
    - 如果一个run里有多个粒子，即多个evnet，那么一个BeginOfRunAction()会对应多个BeginOfEvnetAction()
  - 一个Event中每个step结束就会调用UserSteppingAction()
  - ybsim的能损实现是将step存入map中，然后在对map求和
  - B1RunAction->AddEdep() vs B1EventAction->AddEdep()
    - B1EventAction->AddEdep():多个step的Edep的叠加，单个粒子能损
    - B1RunAction->AddEdep()：多个Event的Edep的叠加，多个粒子能损之和

## B1-多线程

### B1的多线程是什么意思？
- 线程 vs 进程
- 例如：run 10 gammas
  - 单线程
    - 判断是否为有效step
    - 得到有效step的能量沉积，并且加和给EventAction.fEdep
    - EventAction.fEdep加和给RunAction.fEdep
    - 10个gamma逐次模拟
    - 得到最终的RunAction.fEdep
  - 多线程(**难在同一过程，多线程分步后如何合并**)
    - 同样上述步骤，分为两条线，一条线运行5个gamma，另一条线运行5个gamma
    - 对于EventAction.fEdep没有问题，自己和自己加和即可
    - 但两条线会有两个RunAction.fEdep，二者如何加和呢？
    - GetRunAction中对fEdep做了处理，即定义了两个量，一个为fEdep，一个为fEdep2
    - B1已经写好了合并代码（G4AccumulableManager* accumulableManager）
    - G4有个Merge功能，需要自己写代码将多个线程的变量合并
